In [ ]:
#pip install recommenders


In [ ]:
#print(tf.executing_eagerly())  # should be True now

In [1]:
import kagglehub 
import pandas as pd 
import numpy as np 
import os 
import seaborn as sns
import random 
import matplotlib.pyplot as plt
import ast
import tensorflow as tf 
from sklearn.feature_extraction.text import TfidfVectorizer
from transformers import DistilBertTokenizer, TFDistilBertModel
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Input, Embedding, Flatten, Concatenate, Dense, Dropout,Attention,Multiply,GlobalAveragePooling1D
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.preprocessing import MultiLabelBinarizer, MinMaxScaler
from tensorflow.keras.layers import TextVectorization
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from math import sqrt
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Lambda
from tensorflow.keras.optimizers import Adam
from time import time
#from recommenders.evaluation.python_evaluation import (
    #map, ndcg_at_k, precision_at_k, recall_at_k
#)
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm 
from collections import defaultdict,Counter

#### STEP 1 : LOADING THE DATASET FROM KAGGLE 

In [ ]:
devices = tf.config.get_visible_devices()
print(devices)

In [ ]:
strategy = tf.distribute.MirroredStrategy()

In [ ]:
#path = kagglehub.dataset_download("samlearner/letterboxd-movie-ratings-data")

#print("Path to dataset files:", path)

In [ ]:
#movie_data_filename = 'movie_data.csv'
#ratings_filename = 'ratings_export.csv'
#users_filename = 'users_export.csv'
#movie_data_path = os.path.join(path,movie_data_filename)
#ratings_path = os.path.join(path,ratings_filename)
#users_path = os.path.join(path,users_filename)

In [ ]:
# movie_data_df = pd.read_csv(movie_data_path,engine='python')
# ratings_df = pd.read_csv(ratings_path,engine='python')
# users_df = pd.read_csv(users_path,engine = 'python')

In [2]:
movie_data_df = pd.read_parquet('/kaggle/input/parquet-dataset/movie_data.parquet',engine='pyarrow')
ratings_df = pd.read_parquet('/kaggle/input/parquet-dataset/ratings_data.parquet',engine='pyarrow')
users_df = pd.read_parquet('/kaggle/input/parquet-dataset/users_dt.parquet',engine='pyarrow')

In [3]:
def parse_genre_string(genre_str):
    # If the value is not a string (it could be float), convert to '[]'
    if not isinstance(genre_str, str):
        genre_str = '[]'
    # Now attempt parsing
    try:
        parsed = ast.literal_eval(genre_str)
        # If parsing somehow doesn't return a list, force it
        if not isinstance(parsed, list):
            return []
        return parsed
    except Exception:
        return []

In [4]:
movie_data_df['genres'] = movie_data_df['genres'].fillna('[]')
movie_data_df['genres'] = movie_data_df['genres'].apply(parse_genre_string)

In [ ]:
#print(movie_data_df['genres'].head(50))
#print(movie_data_df['genres'].tail(50))
#print(movie_data_df['genres'].dtype)

In [5]:
all_genres1 = set()
for g_list in movie_data_df['genres']:
    all_genres1.update(g_list)

print("Unique genres found:", all_genres1)
print("Number of unique genres:", len(all_genres1))

Unique genres found: {'Crime', 'TV Movie', 'Music', 'Family', 'Comedy', 'Mystery', 'Horror', 'Action', 'Animation', 'Adventure', 'Science Fiction', 'Fantasy', 'Documentary', 'Drama', 'Western', 'Thriller', 'War', 'Romance', 'History'}
Number of unique genres: 19


In [ ]:
print(users_df.head())


# this dataframe won't be used, as we don't want the models predictions to depend on the users data. 

unneccessary columns need to be removed - image url, imdb id, imdb, link (perhaps needed to enhance model with imdb data ? ), tmbd id, link, (download that dataset and enhance with it ? )production countries, 

In [ ]:
print(movie_data_df.info())


# we can check if adding additional information from this dataframe to the model improves its performance (genre of the movie, overview? tokenized, year_released)

In [ ]:
print(ratings_df.head())

In [ ]:
print(ratings_df.info())

In [ ]:
for col in movie_data_df.select_dtypes(include='object'):
    print(f"{col}: {movie_data_df[col].nunique()} unique values")


In [ ]:
#movie_data_df.to_parquet('movie_data.parquet', engine='pyarrow')
#ratings_df.to_parquet('ratings_data.parquet', engine = 'pyarrow')
#users_df.to_parquet('users_dt.parquet',engine = 'pyarrow')
# Load from Parquet file



#### STEP 2: EXPLORATORY DATA ANALYSIS 

In [6]:
movie_data_df = movie_data_df.dropna(subset=['overview'])

In [7]:
movie_data_df['overview_length'] = movie_data_df['overview'].apply(len)

avg_length = movie_data_df['overview_length'].mean()
min_length = movie_data_df['overview_length'].min()
max_length = movie_data_df['overview_length'].max()
movie_data_df.drop(['overview_length'],axis = 1)
print(f"Average Overview Length: {avg_length:.2f}")
print(f"Minimum Overview Length: {min_length}")
print(f"Maximum Overview Length: {max_length}")


Average Overview Length: 283.60
Minimum Overview Length: 1
Maximum Overview Length: 1000


#### STEP 3: DEFINING THE TASK, EVALUATION METRICS, CONSTRUCTING MODEL 

In [ ]:
#tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
#model = TFDistilBertModel.from_pretrained('distilbert-base-uncased')


In [ ]:
#ncf_model.load_weights('ncf_model.weights.h5')
# model can be saved locally after training so it doesn't have to be redone again 

In [ ]:
#texts = movie_data_df['overview'].fillna('').tolist()
#inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="tf", max_length=128)
#with strategy.scope():
    #outputs = model(**inputs).last_hidden_state.mean(dim=1)

# Get embeddings (use outputs.last_hidden_state or outputs.pooler_output)
#movie_df['embedding'] = embeddings.numpy().tolist()

In [8]:
ratings_df.dropna(subset=['user_id', 'movie_id', 'rating_val'], inplace=True)
movie_data_df.dropna(subset=['genres'], inplace=True)

In [ ]:
ratings_df.head()

# non integer frames must be mapped to a unique numeric value - movie id and user id in this case

In [9]:
user_mapping_movie_data = {user : idx for idx, user in enumerate(movie_data_df['movie_id'].unique())}
user_mapping = {user: idx for idx, user in enumerate(ratings_df['user_id'].unique())}
item_mapping = {item: idx for idx, item in enumerate(ratings_df['movie_id'].unique())}
all_genres = set(genre for genres in movie_data_df['genres'] for genre in genres)
genre_mapping = {genre: idx for idx, genre in enumerate(sorted(all_genres))}
movie_data_df['genre_mapped'] = movie_data_df['genres'].apply(
     lambda genres: [genre_mapping[genre] for genre in genres if genre in genre_mapping]
 )
print(genre_mapping)

{'Action': 0, 'Adventure': 1, 'Animation': 2, 'Comedy': 3, 'Crime': 4, 'Documentary': 5, 'Drama': 6, 'Family': 7, 'Fantasy': 8, 'History': 9, 'Horror': 10, 'Music': 11, 'Mystery': 12, 'Romance': 13, 'Science Fiction': 14, 'TV Movie': 15, 'Thriller': 16, 'War': 17, 'Western': 18}


In [ ]:
sample_items = list(user_mapping.items())[:10]
print(sample_items)

In [ ]:
sample_items1 = list(user_mapping_movie_data.items())[:10]
print(sample_items1)

In [10]:
movie_data_df['movie_id'] = movie_data_df['movie_id'].map(user_mapping_movie_data)
print(movie_data_df.head())

                        _id              genres  \
0  5fc85f606758f69634496fd3  [Music, Animation]   
1  5fc85ff26758f696344ace0c                  []   
2  5fc85f606758f69634496fcd             [Drama]   
3  5fc85f606758f69634496fd1             [Drama]   
4  5fc85f606758f69634496fd4       [Documentary]   

                                           image_url    imdb_id  \
0  film-poster/4/6/4/4/4/0/464440-football-freaks...       None   
1  film-poster/2/4/5/5/0/0/245500-aftermath-0-230...  tt0586129   
2  film-poster/9/3/3/1/8/93318-where-chimneys-are...  tt0045731   
3                                               None  tt0187327   
4  film-poster/4/5/4/6/0/3/454603-50-years-of-fab...  tt4769914   

                                         imdb_link  movie_id  \
0                                             None         0   
1  http://www.imdb.com/title/tt0586129/maindetails         1   
2  http://www.imdb.com/title/tt0045731/maindetails         2   
3  http://www.imdb.com/title/tt018

In [11]:
ratings_df['user_id'] = ratings_df['user_id'].map(user_mapping)
ratings_df['movie_id'] = ratings_df['movie_id'].map(item_mapping)
ratings_df.head()

,_id,movie_id,rating_val,user_id
0,5fc57c5d6758f6963451a07f,0,7,0
1,5fc57c5d6758f6963451a063,1,7,0
2,5fc57c5d6758f6963451a0ef,2,7,0
3,5fc57c5d6758f6963451a060,3,4,0
4,5fc57c5c6758f69634519398,4,5,0


In [24]:
#movie_data_df['movie_id'] = movie_data_df['movie_id'].map(user_mapping_movie_data)

In [ ]:
print(movie_data_df.info())

In [ ]:
#movie_data_df = movie_data_df.drop(['genre'],axis = 1)

In [12]:
ratings_df = ratings_df.drop(['_id'],axis = 1)
ratings_df.head()

,movie_id,rating_val,user_id
0,0,7,0
1,1,7,0
2,2,7,0
3,3,4,0
4,4,5,0


In [100]:
train, val = train_test_split(ratings_df, test_size=0.2, random_state=42)
train, test = train_test_split(train, test_size = 0.2, random_state = 42)

In [13]:
n_users = len(user_mapping)
n_items = len(item_mapping)
print(f'No. users : {n_users}, no. items : {n_items}')
#print(user_mapping)

No. users : 7477, no. items : 286070


# easiest to construct model using functional API (multiple input network). 3 dense layers of sizes 64,32,16 respectively follow the embedding layers

In [ ]:
embedding_dim = 32  # Latent factor size
mlp_layer_sizes = [64, 32, 16]  # Fully connected layers

# 1d input for user and item 
user_input = Input(shape=(1,), name='user_input')
item_input = Input(shape=(1,), name='item_input')

# Embedding layers
user_embedding = Embedding(n_users, embedding_dim, name='user_embedding')(user_input)
item_embedding = Embedding(n_items, embedding_dim, name='item_embedding')(item_input)

# Flatten embeddings
user_vec = Flatten()(user_embedding)
item_vec = Flatten()(item_embedding)

gmf_vec = Multiply()([user_vec,item_vec])
# Concatenate embeddings
concat_vec = Concatenate()([user_vec, item_vec])
# MLP layers
mlp = concat_vec
for size in mlp_layer_sizes:
    mlp = Dense(size, activation='relu')(mlp)
    mlp = Dropout(0.2)(mlp)

pre_output_concatenate = Concatenate()([gmf_vec,mlp])
output = Dense(1, activation='linear', name='output')(pre_output_concatenate)

# Build and compile the model
ncf_model = Model(inputs=[user_input, item_input], outputs=output)
ncf_model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

ncf_model.summary()


In [ ]:
#tf.config.get_visible_devices()

In [ ]:
train_dict = defaultdict(set)

# Assuming your train DataFrame has columns 'user_id' and 'movie_id':
for user, item in zip(train['userID'], train['itemID']):
    train_dict[user].add(item)
    

In [ ]:
x_train = [train['user_id'].values, train['movie_id'].values]
y_train = train['rating_val'].values
x_val = [val['user_id'].values, val['movie_id'].values]
y_val = val['rating_val'].values
x_test = [test['user_id'].values, test['movie_id'].values]
y_test = test['rating_val'].values
print(y_test.shape)
print(y_val.shape)
print(type(x_train))
print(type(x_test))
early_stop_callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    min_delta=0,
    patience=0,
    verbose=0,
    mode='auto',
    baseline=None,
    restore_best_weights=True,
    start_from_epoch=0
)
# Train the model
history = ncf_model.fit(
    x=x_train,
    y=y_train,
    batch_size=256,
    epochs=5,
    validation_data=(x_val, y_val),
    verbose=1,
    callbacks = [early_stop_callback]
)
#uncomment above to train model again, weights can be loaded aswell

In [ ]:
#ncf_model.load_weights('ncf_model.weights.h5')

In [ ]:
ncf_model.save_weights('ncf_model.weights.h5')

In [ ]:
loss, mae = ncf_model.evaluate(x_test, y_test)
print(f"Test Loss: {loss:.4f}, Test MAE: {mae:.4f}")

# RMSE
predictions = ncf_model.predict(x_test)
rmse = sqrt(np.mean((predictions.flatten() - y_test) ** 2))
print(f"Test RMSE: {rmse:.4f}")

In [ ]:
user_id = 'jay'  # Replace with a valid user_id
user_idx = user_mapping.get(user_id)
print(user_idx)
# Predict ratings for all items
all_items = np.arange(n_items)
user_tensor = np.array([user_idx] * n_items)
predicted_ratings = ncf_model.predict([user_tensor, all_items])

# Get top 10 recommendations
top_items = np.argsort(predicted_ratings.flatten())[::-1][:10]
recommended_movie_ids = [list(item_mapping.keys())[list(item_mapping.values()).index(idx)] for idx in top_items]
print(f"Recommended movies for user {user_id}: {recommended_movie_ids}")

In [ ]:
test = test.rename(columns={'rating_val': 'prediction'})

In [ ]:
print(test)

In [ ]:
# all_user_ids = list(user_mapping.keys())       # e.g. original user IDs
# all_user_idxs = list(user_mapping.values())    # integer indices
# BATCH_SIZE = 512
# all_items = np.arange(n_items)

# predictions_list = []
# for i in range(0, len(all_user_idxs), BATCH_SIZE):
#     # Take a chunk of users
#     user_idx_batch = all_user_idxs[i : i + BATCH_SIZE]
#     user_id_batch = all_user_ids[i : i + BATCH_SIZE]

#     # Repeat items for each user in the batch
#     # shape: (#users_in_batch * n_items,)
#     tile_users = np.repeat(user_idx_batch, n_items)
#     tile_items = np.tile(all_items, len(user_idx_batch))

#     # Model predict on that entire chunk
#     batch_preds = ncf_model.predict([tile_users, tile_items])  

#     # Now we map predictions back to (user, item) pairs
#     # We'll build a DataFrame
#     df_chunk = pd.DataFrame({
#         "userID": np.repeat(user_id_batch, n_items),
#         "itemID": tile_items,
#         "prediction": batch_preds
#     })
#     predictions_list.append(df_chunk)

# predictions_df = pd.concat(predictions_list, ignore_index=True)
### 1 HOUR 45 MINS FOR ENTIRE DATASET 

In [ ]:
# print(predictions_df)

In [ ]:
test.rename(
    columns={
        "movie_id": "itemID",
        "user_id": "userID",
        "rating_val": "rating"  
    },
    inplace=True,
)

train.rename(
    columns={
        "movie_id": "itemID",  
        "user_id": "userID"
    },
    inplace=True,
)

In [ ]:
# # Evaluate predictions
# #eval_map = map(test, predictions_df, col_prediction='prediction', k=TOP_K)
# eval_ndcg = ndcg_at_k(test, predictions_df, col_prediction='prediction', k=TOP_K)
# print("reaches this")
# eval_precision = precision_at_k(test, predictions_df, col_prediction='prediction', k=TOP_K)
# print("reaches this")
# eval_recall = recall_at_k(test, predictions_df, col_prediction='prediction', k=TOP_K)
# print("reaches this")
# print(
#     #f"MAP:\t{eval_map:.6f}",
#     f"NDCG:\t{eval_ndcg:.6f}",
#     f"Precision@K:\t{eval_precision:.6f}",
#     f"Recall@K:\t{eval_recall:.6f}",
#     sep='\n'
# )

## COMPLEXITY OF PREDICTING FOR ALL USERS AND ITEMS IS (no_users * no_items) -> VERY LARGE. 
Use negative sampling - take 1 item user interacted with, and 50 or 100 items the user did NOT interact (did not rate). 

# with manual testing for different users, the model basically outputs the highest rated shows and movies that the user has not rated. how to improve on that ? -> tokenization of movie descriptions into the model, another pipeline with the genre. 

In [ ]:
print(train)

In [ ]:
print(test)

In [ ]:
all_items_set = set(train['itemID'].unique()) | set(test['itemID'].unique())

In [ ]:
# Parameters
k = 10
hits = []
ndcgs = []
precisions = []
recalls = []
all_true_labels = []
all_pred_scores = []
recommended_items = []

unique_user_ids = test1['userID'].unique()

# Evaluation Loop
for user_id in tqdm(unique_user_ids[:100], desc="Processing users"):
    # Fetch the test items for the user
    test_items = test.loc[test['userID'] == user_id, 'itemID'].values
    pos_items = set(test_items)  
    limited_pos_items = list(pos_items)[:3] # take only 3, on processor only takes too slow for all 
    # Build negative set (exclude train_dict[user_id] + the positive items)
    user_train_items = train_dict[user_id] | set(limited_pos_items)
    possible_negatives = list(all_items_set - user_train_items)

    # Iterate over each positive item
    for pos_item in limited_pos_items:
        # Sample 99 negatives for the current positive item
        neg_sample = random.sample(list(all_items_set - train_dict[user_id] - {pos_item}), 99)
        
        # Combine into a batch
        item_batch = [pos_item] + neg_sample
        user_batch = [user_id] * len(item_batch)
        
        # Get predictions
        scores = ncf_model.predict([np.array(user_batch), np.array(item_batch)])
        scores = np.squeeze(scores)
        
        # Binary labels: 1 for positive item, 0 for negatives
        labels = np.array([1] + [0]*99)
        all_true_labels.extend(labels)
        all_pred_scores.extend(scores)
        
        # Sort items by score in descending order
        sorted_indices = np.argsort(scores)[::-1]
        sorted_labels = labels[sorted_indices]
        sorted_items = np.array(item_batch)[sorted_indices]
        
        # Collect recommended items for distribution analysis
        top_k_items = sorted_items[:k]
        recommended_items.extend(top_k_items.tolist())
        
        # Calculate Hit@K, if positive item in these, its a hit
        if 1 in sorted_labels[:k]:
            hits.append(1)
        else:
            hits.append(0)
        
        # Calculate NDCG@K
        rank = np.where(sorted_labels[:k] == 1)[0]
        if len(rank) > 0:
            ndcgs.append(1 / np.log2(rank[0] + 2))  
        else:
            ndcgs.append(0)
        
        # Calculate Precision@K
        precision = np.sum(sorted_labels[:k]) / k
        precisions.append(precision)
        
        # Calculate Recall@K
        recall = np.sum(sorted_labels[:k]) / len(limited_pos_items)
        recalls.append(recall)

# Calculate Final Metrics
hit_rate = np.mean(hits)
ndcg = np.mean(ndcgs)
precision_at_k = np.mean(precisions)
recall_at_k = np.mean(recalls)
auc = roc_auc_score(all_true_labels, all_pred_scores)



In [ ]:
hit_rate = np.mean(hits)
ndcg = np.mean(ndcgs)
precision_at_k = np.mean(precisions)
recall_at_k = np.mean(recalls)
auc = roc_auc_score(all_true_labels, all_pred_scores)

print(f"Hit Rate@{k}: {hit_rate:.4f}")
print(f"NDCG@{k}: {ndcg:.4f}")
print(f"Precision@{k}: {precision_at_k:.4f}")
print(f"Recall@{k}: {recall_at_k:.4f}")
print(f"ROC AUC: {auc:.4f}")

## Interpretation :
- 45% of users get something relevant to them recommended 
- ndcg : 29% to an ideal ranking system (higher relevance items raise it )
- precision : 1 of 22 items is relevant to user (too low!)
- recall : 15% of a users relevant items are 

In [ ]:
fpr, tpr, thresholds = roc_curve(all_true_labels, all_pred_scores)
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='blue', label=f'ROC curve (AUC = {auc:.4f})')
plt.plot([0, 1], [0, 1], color='red', linestyle='--', label='Random Classifier')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc='lower right')
plt.grid(True)
plt.show()

## Interesting to check the distribution of the recomended movies for each user. 

In [ ]:
item_counts = Counter(recommended_items)
item_counts_df = pd.DataFrame.from_dict(item_counts, orient='index', columns=['count'])
item_counts_df.reset_index(inplace=True)
item_counts_df.rename(columns={'index': 'itemID'}, inplace=True)
item_counts_df.sort_values(by='count', ascending=False, inplace=True)

In [ ]:
plt.figure(figsize=(12, 8))
sns.histplot(item_counts_df['count'], bins=50, kde=True)
plt.xlabel('Number of Recommendations')
plt.ylabel('Frequency')
plt.title('Distribution of Recommended Movies')
plt.show()

In [ ]:
## heavy bias to the highest rated movies (can also be seen by the manual checks for each user)

In [ ]:
# Hit@10: 0.4800
#NDCG@10: 0.3067

In [ ]:
# The relevant items is in 48 percent of reccomended lists to the user, ndcg of 30 percent means we are 30 percent to an ideal ranking 

## This is a good baseline, however it can be improved, in multiple ways

## 1. Train GMF and MLP models separately, then combine into new model, fine-tune that one to see if improvements (paper suggests this technique gives better results)

In [97]:
user_gmf_embedding = Embedding(n_users, embedding_dim, name='user_gmf_embedding')(user_input)
item_gmf_embedding = Embedding(n_items, embedding_dim, name='item_gmf_embedding')(item_input)

# Flatten embeddings
user_gmf_vec = Flatten()(user_gmf_embedding)
item_gmf_vec = Flatten()(item_gmf_embedding)

# Element-wise product of user and item embeddings
gmf_vec = Multiply()([user_gmf_vec, item_gmf_vec])

# Output layer
gmf_output = Dense(1, activation='linear', name='gmf_output')(gmf_vec)

# Build and compile GMF model
gmf_model = Model(inputs=[user_input, item_input], outputs=gmf_output)
gmf_model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

gmf_model.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ user_input (InputLayer)   │ (None, 1)              │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ item_input (InputLayer)   │ (None, 1)              │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ user_gmf_embedding        │ (None, 1, 32)          │        239,264 │ user_input[0][0]       │
│ (Embedding)               │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ item_gmf_embedding        │ (None, 1, 32)          │      9,154,240 │ item_input[0][0]       │
│ (Embedding)               │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ flatten_2 (Flatten)       │ (None, 32)             │              0 │ user_gmf_embedding[0]… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ flatten_3 (Flatten)       │ (None, 32)             │              0 │ item_gmf_embedding[0]… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ multiply (Multiply)       │ (None, 32)             │              0 │ flatten_2[0][0],       │
│                           │                        │                │ flatten_3[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ gmf_output (Dense)        │ (None, 1)              │             33 │ multiply[0][0]         │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 9,393,537 (35.83 MB)

 Trainable params: 9,393,537 (35.83 MB)

 Non-trainable params: 0 (0.00 B)

In [98]:
# MLP model - neural network 
user_mlp_embedding = Embedding(n_users, embedding_dim, name='user_mlp_embedding')(user_input)
item_mlp_embedding = Embedding(n_items, embedding_dim, name='item_mlp_embedding')(item_input)

# Flatten embeddings
user_mlp_vec = Flatten()(user_mlp_embedding)
item_mlp_vec = Flatten()(item_mlp_embedding)

# Concatenate embeddings
mlp_vec = Concatenate()([user_mlp_vec, item_mlp_vec])

# Fully connected layers
mlp = mlp_vec
for size in mlp_layer_sizes:
    mlp = Dense(size, activation='relu')(mlp)
    mlp = Dropout(0.2)(mlp)

# Output layer
mlp_output = Dense(1, activation='linear', name='mlp_output')(mlp)

# Build and compile MLP model
mlp_model = Model(inputs=[user_input, item_input], outputs=mlp_output)
mlp_model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

In [101]:
x_train = [train['user_id'].values, train['movie_id'].values]
y_train = train['rating_val'].values
x_val = [val['user_id'].values, val['movie_id'].values]
y_val = val['rating_val'].values
x_test = [test['user_id'].values, test['movie_id'].values]
y_test = test['rating_val'].values
# Train GMF model
gmf_history = gmf_model.fit(
    x=x_train,
    y=y_train,
    batch_size=256,
    epochs=5,
    validation_data=(x_val, y_val),
    verbose=1
)

# Train MLP model
mlp_history = mlp_model.fit(
    x=x_train,
    y=y_train,
    batch_size=256,
    epochs=5,
    validation_data=(x_val, y_val),
    verbose=1
)

Epoch 1/5
27696/27696 ━━━━━━━━━━━━━━━━━━━━ 99s 4ms/step - loss: 10.9278 - mae: 2.3727 - val_loss: 2.1467 - val_mae: 1.1223
Epoch 2/5
27696/27696 ━━━━━━━━━━━━━━━━━━━━ 96s 3ms/step - loss: 2.0021 - mae: 1.0799 - val_loss: 2.0317 - val_mae: 1.0883
Epoch 3/5
27696/27696 ━━━━━━━━━━━━━━━━━━━━ 96s 3ms/step - loss: 1.7904 - mae: 1.0132 - val_loss: 2.0053 - val_mae: 1.0806
Epoch 4/5
27696/27696 ━━━━━━━━━━━━━━━━━━━━ 96s 3ms/step - loss: 1.6543 - mae: 0.9684 - val_loss: 2.0071 - val_mae: 1.0795
Epoch 5/5
27696/27696 ━━━━━━━━━━━━━━━━━━━━ 97s 3ms/step - loss: 1.5555 - mae: 0.9330 - val_loss: 2.0214 - val_mae: 1.0825
Epoch 1/5
27696/27696 ━━━━━━━━━━━━━━━━━━━━ 111s 4ms/step - loss: 4.1438 - mae: 1.5324 - val_loss: 2.2698 - val_mae: 1.1607
Epoch 2/5
27696/27696 ━━━━━━━━━━━━━━━━━━━━ 105s 4ms/step - loss: 2.2131 - mae: 1.1411 - val_loss: 2.1593 - val_mae: 1.1288
Epoch 3/5
27696/27696 ━━━━━━━━━━━━━━━━━━━━ 105s 4ms/step - loss: 2.0930 - mae: 1.1063 - val_loss: 2.1458 - val_mae: 1.1243
Epoch 4/5
27696/2769

In [102]:
#gmf_model.save_weights('ncf_model.weights.h5')
gmf_model.save_weights('gmf_model.weights.h5')
mlp_model.save_weights('mlp_model.weights.h5')

In [ ]:
alpha = 0.5  

# Weighted combination of GMF and MLP vectors
gmf_weighted = Multiply()([gmf_vec, Lambda(lambda x: x * alpha)(gmf_vec)])
mlp_weighted = Multiply()([mlp, Lambda(lambda x: x * (1 - alpha))(mlp)])

combined = Concatenate()([gmf_weighted, mlp_weighted])

final_output = Dense(1, activation='linear', name='final_output')(combined)

# Build the final NCF model
ncf_combined_model = Model(inputs=[user_input, item_input], outputs=final_output)
ncf_combined_model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

ncf_combined_model.summary()

In [ ]:
alphas = [0.25, 0.5, 0.75]
# trained on all 3, balanced one gave best results 

In [ ]:
# Parameters
k = 10
hits = []
ndcgs = []
precisions = []
recalls = []
all_true_labels = []
all_pred_scores = []
recommended_items = []

unique_user_ids = test['userID'].unique()

# Evaluation Loop
for user_id in tqdm(unique_user_ids[:100], desc="Processing users"):
    # Fetch the test items for the user
    test_items = test.loc[test['userID'] == user_id, 'itemID'].values
    pos_items = set(test_items)  
    limited_pos_items = list(pos_items)[:3] # take only 3, on processor only takes too slow for all 
    # Build negative set (exclude train_dict[user_id] + the positive items)
    user_train_items = train_dict[user_id] | set(limited_pos_items)
    possible_negatives = list(all_items_set - user_train_items)

    # Iterate over each positive item
    for pos_item in limited_pos_items:
        # Sample 99 negatives for the current positive item
        neg_sample = random.sample(list(all_items_set - train_dict[user_id] - {pos_item}), 99)
        
        # Combine into a batch
        item_batch = [pos_item] + neg_sample
        user_batch = [user_id] * len(item_batch)
        
        # Get predictions
        scores = ncf_combined_model.predict([np.array(user_batch), np.array(item_batch)])
        scores = np.squeeze(scores)
        
        # Binary labels: 1 for positive item, 0 for negatives
        labels = np.array([1] + [0]*99)
        all_true_labels.extend(labels)
        all_pred_scores.extend(scores)
        
        # Sort items by score in descending order
        sorted_indices = np.argsort(scores)[::-1]
        sorted_labels = labels[sorted_indices]
        sorted_items = np.array(item_batch)[sorted_indices]
        
        # Collect recommended items for distribution analysis
        top_k_items = sorted_items[:k]
        recommended_items.extend(top_k_items.tolist())
        
        # Calculate Hit@K, if positive item in these, its a hit
        if 1 in sorted_labels[:k]:
            hits.append(1)
        else:
            hits.append(0)
        
        # Calculate NDCG@K
        rank = np.where(sorted_labels[:k] == 1)[0]
        if len(rank) > 0:
            ndcgs.append(1 / np.log2(rank[0] + 2))  
        else:
            ndcgs.append(0)
        
        # Calculate Precision@K
        precision = np.sum(sorted_labels[:k]) / k
        precisions.append(precision)
        
        # Calculate Recall@K
        recall = np.sum(sorted_labels[:k]) / len(limited_pos_items)
        recalls.append(recall)

# Calculate Final Metrics
hit_rate = np.mean(hits)
ndcg = np.mean(ndcgs)
precision_at_k = np.mean(precisions)
recall_at_k = np.mean(recalls)
auc = roc_auc_score(all_true_labels, all_pred_scores)


In [ ]:
hit_rate = np.mean(hits)
ndcg = np.mean(ndcgs)
precision_at_k = np.mean(precisions)
recall_at_k = np.mean(recalls)
auc = roc_auc_score(all_true_labels, all_pred_scores)

print(f"Hit Rate@{k}: {hit_rate:.4f}")
print(f"NDCG@{k}: {ndcg:.4f}")
print(f"Precision@{k}: {precision_at_k:.4f}")
print(f"Recall@{k}: {recall_at_k:.4f}")
print(f"ROC AUC: {auc:.4f}")

In [ ]:
fpr, tpr, thresholds = roc_curve(all_true_labels, all_pred_scores)
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='blue', label=f'ROC curve (AUC = {auc:.4f})')
plt.plot([0, 1], [0, 1], color='red', linestyle='--', label='Random Classifier')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc='lower right')
plt.grid(True)
plt.savefig('good_curve.png')
plt.show()

MAP:	0.044724
NDCG:	0.183073
Precision@K:	0.167020
Recall@K:	0.096622  FROM THE RECOMENDERS PAPER I USED 

Hit Rate@10: 0.6733
NDCG@10: 0.4019
Precision@10: 0.0673
Recall@10: 0.2244
ROC AUC: 0.8771

In [ ]:
#ncf_combined_model.load_weights('ncf_combined_model.weights.h5')

In [ ]:
ncf_combined_model.save_weights('ncf_combined_model.weights.h5')

In [ ]:
print(movie_data_df.info())

In [ ]:
print(merged_df.head())

## 2. Improve on the suggested architecture :
- use other information in the dataset: ex. genre, popularity, overview of the movies 
- add layers to the model - batch normalization, dropout, change loss function
- evaluate the model with tenfold cross validation 
- ensemble with tokenization of movie overview 
- add precision@k,AUC,recall@k,mean reciprocal rank (MRR)
- l1-l2 regularization
- autoencoder of user and item 

In [14]:
movie_data_df = movie_data_df.dropna(subset=['genres', 'popularity','overview'] )

In [ ]:
print(ratings_df)

# turning genres column into one hot encoded columns for each genre. Movies with no genre set should have all zeros in the respective columns. 

In [15]:
max_tokens = 20000 # vocab size 
sequence_length = 300 # output vector length 

In [16]:
mlb = MultiLabelBinarizer()
genres_encoded = mlb.fit_transform(movie_data_df['genres'])
genre_labels = mlb.classes_  # Get genre labels from the MultiLabelBinarizer
print(genre_labels)
genre_labels_cleaned = [c.replace(" ", "_") for c in mlb.classes_]

df_genres = pd.DataFrame(genres_encoded, columns=genre_labels_cleaned)
print(genre_labels_cleaned)
# Now the columns will be "Science_Fiction" rather than "Science Fiction"
movie_data_df = pd.concat([movie_data_df, df_genres], axis=1)

# Also keep your final list to use in the loop
genre_labels = genre_labels_cleaned
scaler = MinMaxScaler()
movie_data_df['popularity_scaled'] = scaler.fit_transform(movie_data_df[['popularity']])
#movie_data_df = pd.concat([movie_data_df, genres_encoded_df], axis=1)
vectorizer = TextVectorization(
    max_tokens=max_tokens,
    output_sequence_length=sequence_length,
    output_mode='int'
)

sample_size = 50000  # adapt on a sample large enough so that the vectorization keeps the most of the context data as possible in each review
sample_overviews = movie_data_df['overview'].dropna().sample(n=sample_size, random_state=42)
overview_ds = tf.data.Dataset.from_tensor_slices(sample_overviews)
vectorizer.adapt(overview_ds.batch(512))  # adapt in batches for memory efficiency


['Action' 'Adventure' 'Animation' 'Comedy' 'Crime' 'Documentary' 'Drama'
 'Family' 'Fantasy' 'History' 'Horror' 'Music' 'Mystery' 'Romance'
 'Science Fiction' 'TV Movie' 'Thriller' 'War' 'Western']
['Action', 'Adventure', 'Animation', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Family', 'Fantasy', 'History', 'Horror', 'Music', 'Mystery', 'Romance', 'Science_Fiction', 'TV_Movie', 'Thriller', 'War', 'Western']


In [17]:
all_overviews = movie_data_df['overview'].fillna("") 

all_overview_ds = tf.data.Dataset.from_tensor_slices(all_overviews).batch(512)

all_overview_ints = []
for batch in all_overview_ds:
    int_batch = vectorizer(batch)  
    all_overview_ints.append(int_batch.numpy())  

all_overview_ints = np.concatenate(all_overview_ints, axis=0)

movie_data_df['overview_ints'] = list(all_overview_ints)

In [18]:
# pretrain a model that embeds the texts and pools them 
text_embedding_dim = 32

text_input = Input(shape=(sequence_length,), dtype='int32', name='text_input')

text_emb_layer = Embedding(input_dim=max_tokens, output_dim=text_embedding_dim, name='text_embedding')
embedded_seq = text_emb_layer(text_input)  # shape: (batch, sequence_length, text_embedding_dim)

overview_embedding = GlobalAveragePooling1D()(embedded_seq)  # shape: (batch, text_embedding_dim)

text_model = Model(inputs=text_input, outputs=overview_embedding)
text_model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ text_input (InputLayer)              │ (None, 300)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ text_embedding (Embedding)           │ (None, 300, 32)             │         640,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling1d             │ (None, 32)                  │               0 │
│ (GlobalAveragePooling1D)             │                             │                 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 640,000 (2.44 MB)

 Trainable params: 640,000 (2.44 MB)

 Non-trainable params: 0 (0.00 B)

In [19]:
overview_ints_dataset = tf.data.Dataset.from_tensor_slices(all_overview_ints).batch(512)

all_embeddings = []
for batch in overview_ints_dataset:
    # Pass the batch through text_model
    emb_batch = text_model(batch, training=False)  # shape: (batch_size, text_embedding_dim)
    all_embeddings.append(emb_batch.numpy())

all_embeddings = np.concatenate(all_embeddings, axis=0)  # shape: (num_movies, text_embedding_dim)


In [20]:
movie_data_df['overview_pre_embed'] = list(all_embeddings)

In [21]:
movie_data_df = movie_data_df.drop(['genres'],axis=1)

In [ ]:
print(movie_data_df.info())

In [ ]:
print(ratings_df.head())

In [22]:
#ratings_df['movie_id'] = ratings_df['movie_id'].astype(int)
#movie_data_df['movie_id'] = movie_data_df['movie_id'].astype(int)
movie_data_df = movie_data_df.dropna(subset=['movie_id'])
ratings_df = ratings_df.dropna(subset=['movie_id'])

In [ ]:
#print(list(mlb.classes_))

In [ ]:
#print(genre_labels)

In [23]:
# Example function to downcast numerics
def downcast_df(df):
    for col in df.select_dtypes(include=['int64', 'float64']).columns:
        if str(df[col].dtype) == 'int64':
            df[col] = pd.to_numeric(df[col], downcast='integer')
        elif str(df[col].dtype) == 'float64':
            df[col] = pd.to_numeric(df[col], downcast='float')
    return df

In [ ]:
print(movie_data_df.info())

In [24]:
ratings_df = downcast_df(ratings_df)
movie_data_df = downcast_df(movie_data_df)

In [25]:
genre_columns = [col for col in movie_data_df.columns if col in genre_labels]
print(genre_columns)

['Action', 'Adventure', 'Animation', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Family', 'Fantasy', 'History', 'Horror', 'Music', 'Mystery', 'Romance', 'Science_Fiction', 'TV_Movie', 'Thriller', 'War', 'Western']


In [26]:
movie_data_df = movie_data_df.drop_duplicates(subset=['movie_id'])

In [65]:
print(ratings_df.info(memory_usage='deep'))
print(movie_data_df.info(memory_usage='deep'))

<class 'pandas.core.frame.DataFrame'>
Index: 11078161 entries, 0 to 11078166
Data columns (total 3 columns):
 #   Column      Dtype
---  ------      -----
 0   movie_id    int32
 1   rating_val  int8 
 2   user_id     int16
dtypes: int16(1), int32(1), int8(1)
memory usage: 158.5 MB
None
<class 'pandas.core.frame.DataFrame'>
Index: 261144 entries, 0 to 285956
Data columns (total 42 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   _id                   261144 non-null  object 
 1   image_url             244615 non-null  object 
 2   imdb_id               227942 non-null  object 
 3   imdb_link             227943 non-null  object 
 4   movie_id              261144 non-null  float32
 5   movie_title           261144 non-null  object 
 6   original_language     261144 non-null  object 
 7   overview              261144 non-null  object 
 8   popularity            261144 non-null  float32
 9   production_countries  261143 

In [27]:
useful_cols = ['movie_id', 'overview_pre_embed','popularity_scaled']+ genre_labels #+ genre_columns, add popularity scaled as well 
movie_data_small = movie_data_df[useful_cols]

merged_df_large = ratings_df.merge(
    movie_data_small,  
    on='movie_id', 
    how='inner'
)


In [67]:
print(merged_df_large.info())

<class 'pandas.core.frame.DataFrame'>
Index: 10874589 entries, 0 to 10874588
Data columns (total 24 columns):
 #   Column              Dtype  
---  ------              -----  
 0   movie_id            int32  
 1   rating_val          int8   
 2   user_id             int16  
 3   overview_pre_embed  object 
 4   popularity_scaled   float32
 5   Action              float32
 6   Adventure           float32
 7   Animation           float32
 8   Comedy              float32
 9   Crime               float32
 10  Documentary         float32
 11  Drama               float32
 12  Family              float32
 13  Fantasy             float32
 14  History             float32
 15  Horror              float32
 16  Music               float32
 17  Mystery             float32
 18  Romance             float32
 19  Science_Fiction     float32
 20  TV_Movie            float32
 21  Thriller            float32
 22  War                 float32
 23  Western             float32
dtypes: float32(20), int16(1), i

In [28]:
merged_df_large = merged_df_large.dropna(subset = genre_labels)

In [29]:
train1, val1 = train_test_split(merged_df_large, test_size=0.2, random_state=42)
train1, test1 = train_test_split(train1, test_size = 0.2, random_state = 42)

In [30]:
X_user = train1['user_id'].values
X_item = train1['movie_id'].values
X_text = np.vstack(train1['overview_pre_embed'].values)  # shape (num_samples, text_embedding_dim)
X_popularity = train1['popularity_scaled'].values.reshape(-1, 1)  # shape (num_samples, 1)
X_genres = train1[genre_labels].values  # shape (num_samples, len(genre_columns))

In [31]:
y = train1['rating_val'].values  

In [71]:
print(val1.info())

<class 'pandas.core.frame.DataFrame'>
Index: 2174918 entries, 8661868 to 6469180
Data columns (total 24 columns):
 #   Column              Dtype  
---  ------              -----  
 0   movie_id            int32  
 1   rating_val          int8   
 2   user_id             int16  
 3   overview_pre_embed  object 
 4   popularity_scaled   float32
 5   Action              float32
 6   Adventure           float32
 7   Animation           float32
 8   Comedy              float32
 9   Crime               float32
 10  Documentary         float32
 11  Drama               float32
 12  Family              float32
 13  Fantasy             float32
 14  History             float32
 15  Horror              float32
 16  Music               float32
 17  Mystery             float32
 18  Romance             float32
 19  Science_Fiction     float32
 20  TV_Movie            float32
 21  Thriller            float32
 22  War                 float32
 23  Western             float32
dtypes: float32(20), int16(1

In [32]:
num_users = X_user.max() + 1
num_items = X_item.max() + 1
num_genres = len(genre_columns)

In [33]:
print("Data shapes:")
print("X_user:", X_user.shape)
print("X_item:", X_item.shape)
print("X_text:", X_text.shape)
print("X_popularity:", X_popularity.shape)
print("X_genres:", X_genres.shape)
print("y:", y.shape)

Data shapes:
X_user: (6959736,)
X_item: (6959736,)
X_text: (6959736, 32)
X_popularity: (6959736, 1)
X_genres: (6959736, 19)
y: (6959736,)


In [34]:
embedding_dim = 32
mlp_layer_sizes = [64, 32, 16]
n_users = 10000
n_items = 5000
num_genres = 19
overview_dim = 32

user_input = Input(shape=(1,), name='user_input')
item_input = Input(shape=(1,), name='item_input')
genre_input = Input(shape=(num_genres,), name='genre_input')
popularity_input = Input(shape=(1,), name='popularity_input')
overview_input = Input(shape=(overview_dim,), name='overview_input')

# ---------------------------
# 2) Embeddings (Reused)
# ---------------------------
user_embedding = Embedding(n_users, embedding_dim, name='user_embedding')
item_embedding = Embedding(n_items, embedding_dim, name='item_embedding')

user_vec = Flatten()(user_embedding(user_input))
item_vec = Flatten()(item_embedding(item_input))

# ---------------------------
# 3) GMF path (element-wise product)
# ---------------------------
gmf_vec = Multiply(name='gmf_interaction')([user_vec, item_vec])

# ---------------------------
# 4) MLP path (concat user_vec, item_vec & item features)
# ---------------------------
# Reduce overview dimension
#overview_reduced = Dense(32, activation='relu', name='overview_reduced')(overview_input)

item_features = Concatenate(name='item_features_concat')([
    item_vec,
    genre_input,
    popularity_input,
    overview_input
])

mlp_input = Concatenate(name='mlp_concat')([
    user_vec,
    item_features
])

mlp = mlp_input
for size in mlp_layer_sizes:
    mlp = Dense(size, activation='relu')(mlp)
    mlp = Dropout(0.2)(mlp)

mlp_vec = mlp

# ---------------------------
# 5) Final: Concatenate GMF & MLP + Output
# ---------------------------
final_concat = Concatenate(name='final_concat')([gmf_vec, mlp_vec])
output = Dense(1, activation='linear', name='output')(final_concat)

ncf_model = Model(
    inputs=[
        user_input,
        item_input,
        genre_input,
        popularity_input,
        overview_input
    ],
    outputs=output
)

# ---------------------------
# 6) Compile & Summary
# ---------------------------
ncf_model.compile(
    optimizer='adam',
    loss='mean_squared_error',
    metrics=['mae']
)

ncf_model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ item_input (InputLayer)   │ (None, 1)              │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ user_input (InputLayer)   │ (None, 1)              │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ item_embedding            │ (None, 1, 32)          │        160,000 │ item_input[0][0]       │
│ (Embedding)               │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ user_embedding            │ (None, 1, 32)          │        320,000 │ user_input[0][0]       │
│ (Embedding)               │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ flatten_1 (Flatten)       │ (None, 32)             │              0 │ item_embedding[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ genre_input (InputLayer)  │ (None, 19)             │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ popularity_input          │ (None, 1)              │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ overview_input            │ (None, 32)             │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ flatten (Flatten)         │ (None, 32)             │              0 │ user_embedding[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ item_features_concat      │ (None, 84)             │              0 │ flatten_1[0][0],       │
│ (Concatenate)             │                        │                │ genre_input[0][0],     │
│                           │                        │                │ popularity_input[0][0… │
│                           │                        │                │ overview_input[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ mlp_concat (Concatenate)  │ (None, 116)            │              0 │ flatten[0][0],         │
│                           │                        │                │ item_features_concat[… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense (Dense)             │ (None, 64)             │          7,488 │ mlp_concat[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout (Dropout)         │ (None, 64)             │              0 │ dense[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_1 (Dense)           │ (None, 32)             │          2,080 │ dropout[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_1 (Dropout)       │ (None, 32)             │              0 │ dense_1[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_2 (Dense)      

 Total params: 490,145 (1.87 MB)

 Trainable params: 490,145 (1.87 MB)

 Non-trainable params: 0 (0.00 B)

In [35]:
X_train = [
    X_user,        # user_input
    X_item,        # item_input
    X_text,        # text_input
    X_popularity,  # popularity_input
    X_genres       # genre_input
]

In [ ]:
# model.fit(
#     x= X_train,
#     y=y,
#     batch_size=256,
#     epochs=5,
#     #validation_split=0.1
#     validation_data = [val1]
# )

In [61]:
print(train1.info())

<class 'pandas.core.frame.DataFrame'>
Index: 6959736 entries, 418802 to 10605548
Data columns (total 24 columns):
 #   Column              Dtype  
---  ------              -----  
 0   movie_id            int32  
 1   rating_val          int8   
 2   user_id             int16  
 3   overview_pre_embed  object 
 4   popularity_scaled   float32
 5   Action              float32
 6   Adventure           float32
 7   Animation           float32
 8   Comedy              float32
 9   Crime               float32
 10  Documentary         float32
 11  Drama               float32
 12  Family              float32
 13  Fantasy             float32
 14  History             float32
 15  Horror              float32
 16  Music               float32
 17  Mystery             float32
 18  Romance             float32
 19  Science_Fiction     float32
 20  TV_Movie            float32
 21  Thriller            float32
 22  War                 float32
 23  Western             float32
dtypes: float32(20), int16(1

In [36]:
print(y_train1.size)

NameError: name 'y_train1' is not defined

In [37]:

# 1) user_id: shape (num_samples,)
user_ids_train = train1['user_id'].values  

# 2) movie_id: shape (num_samples,)
item_ids_train = train1['movie_id'].values

# 3) genre vector: shape (num_samples, num_genres)
genre_train = train1[genre_labels].values  

# 4) popularity: shape (num_samples, 1)
popularity_train = train1['popularity_scaled'].values.reshape(-1, 1)

# 5) overview: shape (num_samples, overview_dim)
overview_train = np.vstack(train1['overview_pre_embed'].values)

# y: shape (num_samples,)
y_train = train1['rating_val'].values

x_train = [
    user_ids_train,       
    item_ids_train,       
    genre_train,          
    popularity_train,     
    overview_train        
]

user_ids_val = val1['user_id'].values
item_ids_val = val1['movie_id'].values
genre_val = val1[genre_labels].values
popularity_val = val1['popularity_scaled'].values.reshape(-1, 1)
overview_val = np.vstack(val1['overview_pre_embed'].values)
y_val = val1['rating_val'].values

x_val = [
    user_ids_val,
    item_ids_val,
    genre_val,
    popularity_val,
    overview_val
]


In [83]:
print(np.isnan(y_train).any())

False


In [77]:
print(np.isnan(overview_train).any())

False


In [78]:
print(np.isnan(popularity_train).any())

False


In [81]:
print(np.isnan(genre_train).any())

False


In [ ]:
#big_model.load_weights('ncf_combined_model.weights.h5')
# try without loading the pretrained first 

In [ ]:
print(train.info())

In [ ]:
print(train1.info())

In [38]:
history = ncf_model.fit(
    x=x_train,
    y=y_train,
    batch_size=256,
    epochs=10,
    validation_data=(x_val, y_val),
    verbose=1
)

Epoch 1/10
27187/27187 ━━━━━━━━━━━━━━━━━━━━ 75s 3ms/step - loss: 4.2812 - mae: 1.5949 - val_loss: 2.9028 - val_mae: 1.3102
Epoch 2/10
27187/27187 ━━━━━━━━━━━━━━━━━━━━ 66s 2ms/step - loss: 2.8952 - mae: 1.3107 - val_loss: 2.8361 - val_mae: 1.2912
Epoch 3/10
27187/27187 ━━━━━━━━━━━━━━━━━━━━ 66s 2ms/step - loss: 2.8150 - mae: 1.2872 - val_loss: 2.8040 - val_mae: 1.2858
Epoch 4/10
27187/27187 ━━━━━━━━━━━━━━━━━━━━ 66s 2ms/step - loss: 2.7677 - mae: 1.2726 - val_loss: 2.7940 - val_mae: 1.2832
Epoch 5/10
27187/27187 ━━━━━━━━━━━━━━━━━━━━ 65s 2ms/step - loss: 2.7269 - mae: 1.2619 - val_loss: 2.7773 - val_mae: 1.2785
Epoch 6/10
27187/27187 ━━━━━━━━━━━━━━━━━━━━ 66s 2ms/step - loss: 2.7006 - mae: 1.2542 - val_loss: 2.7727 - val_mae: 1.2760
Epoch 7/10
27187/27187 ━━━━━━━━━━━━━━━━━━━━ 66s 2ms/step - loss: 2.6790 - mae: 1.2472 - val_loss: 2.7686 - val_mae: 1.2752
Epoch 8/10
27187/27187 ━━━━━━━━━━━━━━━━━━━━ 66s 2ms/step - loss: 2.6603 - mae: 1.2420 - val_loss: 2.7698 - val_mae: 1.2739
Epoch 9/10
27187

In [39]:
ncf_model.save_weights('ncf_model_big.weights.h5')

In [40]:
train_dict1 = defaultdict(set)

# Assuming your train DataFrame has columns 'user_id' and 'movie_id':
for user, item in zip(train1['user_id'], train1['movie_id']):
    train_dict1[user].add(item)
    

In [41]:
all_items_set = set(train1['movie_id'].unique()) | set(test1['movie_id'].unique())

In [42]:
item_overview_map = dict(zip(movie_data_df['movie_id'], movie_data_df['overview_pre_embed']))

In [44]:
item_genre_map = {}
item_popularity_map = {}
item_overview_map = {}
test_item_ids = set(test1['movie_id'].unique())

sub_df = merged_df_large[merged_df_large['movie_id'].isin(test_item_ids)]

for idx, row in tqdm(sub_df.iterrows(), 
                     total=len(sub_df), 
                     desc="Processing test items"):
    item_id = row["movie_id"]
    genre_vec = np.array([row[g] for g in genre_labels], dtype=np.float32)

    # Suppose you have genre columns g1...g19 in your DF
    # Create the genre vector:
    # If you stored them as separate columns, collect them:
    
    # Popularity
    popularity_val = row.popularity_scaled  # or getattr(row, 'popularity_scaled')

    # Overview pre-embedding (if stored as a list in the DF, you might do something else)
    # If you stored them as an actual Python list or array in the DataFrame, just reference it
    overview_vec = np.array(row.overview_pre_embed, dtype=np.float32)

    item_genre_map[item_id] = genre_vec
    item_popularity_map[item_id] = popularity_val
    item_overview_map[item_id] = overview_vec


Processing test items: 100%|██████████| 10584621/10584621 [22:32<00:00, 7824.34it/s]


In [84]:
print(test1[genre_labels].isna().sum())

Action             0
Adventure          0
Animation          0
Comedy             0
Crime              0
Documentary        0
Drama              0
Family             0
Fantasy            0
History            0
Horror             0
Music              0
Mystery            0
Romance            0
Science_Fiction    0
TV_Movie           0
Thriller           0
War                0
Western            0
dtype: int64


In [87]:
print(test1.head())

          movie_id  rating_val  user_id  \
2601918       1720           9     1891   
9563063      51579           5     1062   
8663830      27139           9     1464   
10127365     97121           5     2629   
5588477       5645           4     1240   

                                         overview_pre_embed  \
2601918   [0.029842695, -0.014418669, -0.02784736, 0.010...   
9563063   [0.028368186, -0.013414619, -0.027028503, 0.00...   
8663830   [0.03683201, -0.017867621, -0.03552789, 0.0139...   
10127365  [0.029536689, -0.014158136, -0.028115228, 0.01...   
5588477   [0.032639775, -0.015718715, -0.030518478, 0.01...   

          popularity_scaled  Action  Adventure  Animation  Comedy  Crime  ...  \
2601918             0.00010     0.0        0.0        0.0     0.0    0.0  ...   
9563063             0.00010     0.0        0.0        0.0     1.0    0.0  ...   
8663830             0.00010     0.0        0.0        0.0     0.0    0.0  ...   
10127365            0.00205     0.0   

In [86]:
#print(movie_data_df[genre_labels].isna().sum())

In [46]:

k = 10
hits = []
ndcgs = []
precisions = []
recalls = []
all_true_labels = []
all_pred_scores = []
recommended_items = []

unique_user_ids = test1['user_id'].unique()

all_items_set = set(movie_data_df['movie_id'])  

for user_id in tqdm(unique_user_ids[:100], desc="Processing users"):
    # 1) Fetch the test (positive) items for this user - he rated (interacted)
    test_items = test1.loc[test1['user_id'] == user_id, 'movie_id'].values
    pos_items = set(test_items)

    # If you only want to evaluate, say, 3 positives:
    limited_pos_items = list(pos_items)[:3]

    # 2) Retrieve training items for user (to exclude them in negative sampling)
    #    Assuming you have a dict: train_dict[user_id] = set of items user has interacted with in train
    user_train_items = train_dict1[user_id] | set(limited_pos_items)
    
    # 3) Negative sampling
    possible_negatives = all_items_set - user_train_items

    for pos_item in limited_pos_items:
        # Sample 99 negatives
        neg_sample = random.sample(list(possible_negatives - {pos_item}), 99)

        # item_batch are the actual item IDs
        item_batch = [pos_item] + neg_sample

        # user_batch is the user repeated
        user_batch = [user_id] * len(item_batch)
        item_batch_np = np.array(item_batch, dtype=np.int32)
        item_batch = [int(i) for i in item_batch]
        # 2) genre_batch_np
        genre_batch = [item_genre_map[i] for i in item_batch]
        genre_batch_np = np.array(genre_batch, dtype=np.float32)
        
        # 3) popularity_batch_np
        pop_batch = [item_popularity_map[i] for i in item_batch]
        popularity_batch_np = np.array(pop_batch, dtype=np.float32).reshape(-1, 1)
        
        # 4) overview_batch_np
        ov_batch = [item_overview_map[i] for i in item_batch]
        overview_batch_np = np.array(ov_batch, dtype=np.float32)
        # 4) Build item_emb_batch by looking up each item ID in the embedding map
        #item_emb_batch = [item_overview_map[item] for item in item_batch]
        # item_emb_batch is now a list of arrays (each shape = (text_embedding_dim,))

        # Convert to NumPy
        user_batch_np = np.array(user_batch, dtype=np.int32)         # shape: (batch_size,)
        #item_emb_batch_np = np.array(item_emb_batch, dtype=np.float32)  # shape: (batch_size, text_embedding_dim)

        # 5) Predict
        scores = ncf_model.predict([
        user_batch_np,
        item_batch_np,
        genre_batch_np,
        popularity_batch_np,
        overview_batch_np
        ], verbose=0)
        scores = np.squeeze(scores) 

        # 6) Prepare labels (1 positive, 99 negatives)
        labels = np.array([1] + [0]*99)
        all_true_labels.extend(labels)
        all_pred_scores.extend(scores)

        # 7) Sort items by predicted score (descending)
        sorted_indices = np.argsort(scores)[::-1]
        sorted_labels = labels[sorted_indices]
        sorted_items = np.array(item_batch)[sorted_indices]

        # 8) Evaluate metrics
        # Hit@K
        if 1 in sorted_labels[:k]:
            hits.append(1)
        else:
            hits.append(0)

        # NDCG@K
        rank = np.where(sorted_labels[:k] == 1)[0]
        if len(rank) > 0:
            ndcgs.append(1 / np.log2(rank[0] + 2))  
        else:
            ndcgs.append(0)

        # Precision@K
        precision = np.sum(sorted_labels[:k]) / k
        precisions.append(precision)

        # Recall@K
        recall = np.sum(sorted_labels[:k]) / len(limited_pos_items)
        recalls.append(recall)

# 9) Compute final metrics
hit_rate = np.mean(hits)
ndcg = np.mean(ndcgs)
precision_at_k = np.mean(precisions)
recall_at_k = np.mean(recalls)
auc = roc_auc_score(all_true_labels, all_pred_scores)

print(f"Hit@{k}: {hit_rate}")
print(f"NDCG@{k}: {ndcg}")
print(f"Precision@{k}: {precision_at_k}")
print(f"Recall@{k}: {recall_at_k}")
print(f"AUC: {auc}")

Processing users:   0%|          | 0/100 [00:00<?, ?it/s]


KeyError: 180449

In [54]:
print("Any NaN in user_batch_np?", np.isnan(user_batch_np).any())
print("Any NaN in item_batch_np?", np.isnan(item_batch_np).any())
print("Any NaN in genre_batch_np?", np.isnan(genre_batch_np).any())
print("Any NaN in popularity_batch_np?", np.isnan(popularity_batch_np).any())
print("Any NaN in overview_batch_np?", np.isnan(overview_batch_np).any())

Any NaN in user_batch_np? False
Any NaN in item_batch_np? False
Any NaN in genre_batch_np? True
Any NaN in popularity_batch_np? False
Any NaN in overview_batch_np? False


In [ ]:
fpr, tpr, thresholds = roc_curve(all_true_labels, all_pred_scores)
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='blue', label=f'ROC curve (AUC = {auc:.4f})')
plt.plot([0, 1], [0, 1], color='red', linestyle='--', label='Random Classifier')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc='lower right')
plt.grid(True)
plt.show()

In [ ]:
print(test1.info())

### 3. Pretrained GMF,MLP and overview based model combined together. 

In [104]:
user_embedding_dim = 32

In [105]:
user_input = Input(shape=(1,), dtype='int32', name='user_input')
text_input = Input(shape=(sequence_length,), dtype='int32', name='text_input')

# ---------------------------
# 2) User Embedding
# ---------------------------
user_embedding_layer = Embedding(
    input_dim=n_users, 
    output_dim=user_embedding_dim, 
    name='user_embedding'
)
user_emb = user_embedding_layer(user_input)  # shape (None, 1, user_embedding_dim)
user_vec = Flatten()(user_emb)               # shape (None, user_embedding_dim)

# ---------------------------
# 3) Text Embedding
# ---------------------------
text_embedding_layer = Embedding(
    input_dim=max_tokens, 
    output_dim=text_embedding_dim, 
    name='text_embedding'
)
text_embedded_seq = text_embedding_layer(text_input)  
# shape (None, sequence_length, text_embedding_dim)

# Global average pooling to get a single vector
text_vec = GlobalAveragePooling1D()(text_embedded_seq)
# shape (None, text_embedding_dim)

# ---------------------------
# 4) Combine User + Text
# ---------------------------
# Simple approach: concatenate user vector & text vector, then MLP
combined_vec = Concatenate()([user_vec, text_vec])  # shape (None, user_embedding_dim + text_embedding_dim)

x = Dense(64, activation='relu')(combined_vec)
x = Dropout(0.2)(x)
x = Dense(32, activation='relu')(x)
x = Dropout(0.2)(x)

# Final rating output
output = Dense(1, activation='linear', name='rating_output')(x)

# ---------------------------
# 5) Build & Compile
# ---------------------------
text_only_cf_model = Model(
    inputs=[user_input, text_input],
    outputs=output
)

text_only_cf_model.compile(
    optimizer='adam',
    loss='mse',
    metrics=['mae']
)

text_only_cf_model.summary()

Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ user_input (InputLayer)   │ (None, 1)              │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ text_input (InputLayer)   │ (None, 300)            │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ user_embedding            │ (None, 1, 32)          │        239,264 │ user_input[0][0]       │
│ (Embedding)               │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ text_embedding            │ (None, 300, 32)        │        640,000 │ text_input[0][0]       │
│ (Embedding)               │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ flatten_6 (Flatten)       │ (None, 32)             │              0 │ user_embedding[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ global_average_pooling1d… │ (None, 32)             │              0 │ text_embedding[0][0]   │
│ (GlobalAveragePooling1D)  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate_1             │ (None, 64)             │              0 │ flatten_6[0][0],       │
│ (Concatenate)             │                        │                │ global_average_poolin… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_6 (Dense)           │ (None, 64)             │          4,160 │ concatenate_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_6 (Dropout)       │ (None, 64)             │              0 │ dense_6[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_7 (Dense)           │ (None, 32)             │          2,080 │ dropout_6[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_7 (Dropout)       │ (None, 32)             │              0 │ dense_7[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ rating_output (Dense)     │ (None, 1)              │             33 │ dropout_7[0][0]        │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 885,537 (3.38 MB)

 Trainable params: 885,537 (3.38 MB)

 Non-trainable params: 0 (0.00 B)

In [40]:
useful_cols = ['movie_id', 'overview']
movie_data_small = movie_data_df[useful_cols]
merged_df = ratings_df.merge(
    movie_data_small,  
    on='movie_id', 
    how='inner'
)

In [47]:
print(merged_df)

          movie_id  rating_val  user_id  \
0                0           7        0   
1                0          10        4   
2                0           8       10   
3                0          10       17   
4                0           8       20   
...            ...         ...      ...   
10970327    261255           9     2539   
10970328    261256           8     5423   
10970329    261256           6     4505   
10970330    261256           1     7046   
10970331    261257           9     5423   

                                                   overview  
0         Football crazy, football mad. Don’t watch this...  
1         Football crazy, football mad. Don’t watch this...  
2         Football crazy, football mad. Don’t watch this...  
3         Football crazy, football mad. Don’t watch this...  
4         Football crazy, football mad. Don’t watch this...  
...                                                     ...  
10970327  Internet icons, teamed up with JibJab a

In [42]:
X = merged_df['overview']

y = merged_df['rating_val']

In [43]:
X_train, X_test, y_train, y_test = train_test_split(
    X, 
    y, 
    test_size=0.2,     # 20% of the data for testing (adjust as needed)
    random_state=42    # for reproducibility
)


In [ ]:
vectorizer = TfidfVectorizer()
sample_size = 50000
sample_df = merged_df.sample(n=sample_size, random_state=42)
#sample_df = shuffle(sample_df, random_state=42)
# Fit vectorizer on the sample text and transform the training text 
vectorizer.fit(sample_df)
X_train_tfidf = vectorizer.transform(X_train)

# Transform (using the trained vocabulary) on the test text
X_test_tfidf = vectorizer.transform(X_test)